# Информационный поиск и парсинг

1) Регулярные выражения  
Фразовые запросы и ранжированный информационный поиск;  
Методы оценки качества поисковых машин.  
Синтаксис составляющих и синтаксис зависимостей;  
Контекстонезависимые грамматики;  
Вероятностный подход к парсингу;  
Лексикализованные вероятностные грамматики;  
Применение парсинга в различных задачах.

### 1) Регулярные выражения  

При работе с текстами часто приходится решать задачу поиска в тексте по шаблону, например в текстовом редакторе или в браузере. 
Формализуем задачу: задается текстовая строка длиной $N$ и шаблон длиной $M$, нужно найти индекс первого вхождения шаблона в текст. Большинство алгоритмов для этой задачи можно легко расширить, чтобы найти все вхождения
шаблона в тексте, подсчитать количество вхождений шаблона в тексте или предоставить контекст (подстроки текста, окружающие каждое вхождение шаблона). Поиск подстроки - классическая задача, которая имеет множество интересных решений, ознакомиться с которыми можно в книге Р.Сэджвика Алгоритмы, 5.3. 

Однако сегодня мы поговорим о поиске на основании неполной информации об искомом образце. Все возможные слова, подходящие под искомый шаблон, задают язык искомых слов. Для поиска вхождений шаблона в заданном тексте используются регулярные выражения.  

**Регулярное выражение** (regexp, регулярка) - это шаблон, сопоставляемый с искомой строкой слева направо. 
Регулярки описывают образцы с помощью трех простых, но мощных операций: 
- конкатенация,  
- логическое или,  
- замыкание.  

Конкатенация позволяет определить язык (искомых слов) путем простой записи символов рядом друг с другом. Например, запись *AB* определяет язык *{AB}*, содержащий всего одно слово.  

Логическое ИЛИ позволяет определить несколько слов в языке. Например, запись *AB|CD* означает что нужно искать слова *AB* или *CD*.  

Замыкание позволяет повторять части образца произвольное число раз, в т.ч. ни разу. Например, регулярка AB*  задает слова {B, AB, AAB, AAAB ...} а регулярка *A\*B* - *A, AB, ABB, ABBB, ...*  

Для определения порядка подстановки используются круглые скобки. Например, выражение C(AC|B)D задает множество слов {CACD, CBD}, а регулярка (AB)* - {$\in$ , AB, ABAB, ABABAB, ...} где $\in$ - пустая строка.  

Стандартная библиотека содержит модуль re для работы с ASCII алфавитом. Для работы с Unicode алфавитом используется сторонний модуль regexp.   

Метод match() объекта re.Pattern проверяет совпадение строки с регуляркой. Совпадением считается только соответствие шаблону начала строки. В результате возвращается объект re.Match, в котором содержатся span - (индекс начала, длина совпадения) и match - совпавшая строка. 

In [1]:
import re

p = re.compile('a*') # pattern
sent = ['abababa',
        'aaaaaab',
        'bbbbbbb',
        'bababab'
       ]
        
for s in sent:
    print(p.match(s))

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 6), match='aaaaaa'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>


Также можно использовать top-level методы из модуля re. Шаблон можно не компилировать, достаточно указать символ r перед строкой с одинарными кавычками. 

In [2]:
for s in sent:
    print(re.match(r'ab*', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
None


In [3]:
# Метод search() ищет совпадения по всей строке.
for s in sent:
    print(re.search(r'ab*', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
<re.Match object; span=(1, 3), match='ab'>


Задание: Напишите пример использования методов findall() и sub().

Кроме того, в регулярках можно использовать метасимволы:  
. - (dot) любой символ, кроме \n

In [4]:
for s in sent:
    print(re.match(r'.', s))

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='b'>
<re.Match object; span=(0, 1), match='b'>


^ - (caret) начало строки

In [5]:
for s in sent:
    print(re.match(r'^b', s))

None
None
<re.Match object; span=(0, 1), match='b'>
<re.Match object; span=(0, 1), match='b'>


$ - конец строки

In [6]:
for s in sent:
    print(re.search(r'a$', s))

<re.Match object; span=(6, 7), match='a'>
None
None
None


\+ - не меньше одного вхождения, в отличии от *, который допускает ни одного.

In [7]:
for s in sent:
    print(re.match(r'b+', s))

None
None
<re.Match object; span=(0, 7), match='bbbbbbb'>
<re.Match object; span=(0, 1), match='b'>


? - 0 или 1 вхождение

In [8]:
for s in sent:
    print(re.search(r'ab?', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
<re.Match object; span=(1, 3), match='ab'>


{m} - определяет m количество вхождений

In [9]:
for s in sent:
    print(re.search(r'b{3}', s))

None
None
<re.Match object; span=(0, 3), match='bbb'>
None


{m,n} - количество вхождений от m до n

In [10]:
for s in sent:
    print(re.search(r'b{3,5}', s))

None
None
<re.Match object; span=(0, 5), match='bbbbb'>
None


{m,n}? - жадное (минимальное) количество вхождений от m до n

In [11]:
for s in sent:
    print(re.search(r'b{3,5}?', s))

None
None
<re.Match object; span=(0, 3), match='bbb'>
None


\ - обратный слэш, экранирует спецсимволы

In [12]:
s = '***'
re.match(r'\*', s)

<re.Match object; span=(0, 1), match='*'>

[ ] - определяет множество символов

In [13]:
for s in sent:
    print(re.match(r'[ab]a', s))

None
<re.Match object; span=(0, 2), match='aa'>
None
<re.Match object; span=(0, 2), match='ba'>


[a-z] - определяет диапазон символов

In [15]:
s = 'bcde'
re.match(r'[a-c]', s)

<re.Match object; span=(0, 1), match='b'>

In [16]:
re.match(r'[0-5][0-9]', '59')

<re.Match object; span=(0, 2), match='59'>

Ссылки:  

- Сэджвик Р. и др., Алгоритмы, 
- https://docs.python.org/3/library/re.html
- https://docs.python.org/3/howto/regex.html
- https://habr.com/ru/post/349860/